# <font color = blue> Capstone Project for Data Science Certification </font>

In [1]:
## Introduction

This notebook is intended for completing the Capstone Project, required for the IBM Data Science Certification. This is being completed by Bad Scooter.

In [9]:
import pandas as pd
import numpy as np
from urllib.request import urlopen #Used to open the url that contains our data
from bs4 import BeautifulSoup #The BeautifulSoup class contains the methods necessary to extract the data from the web site (url)

In [3]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Peer-graded Assignment Part One

In [43]:
# 1. Create a Dataframe by web scraping (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_can = requests.get(url) #Return the html from the web page
soup = BeautifulSoup(raw_can.content, 'lxml') #Create an object which is parsable
table = soup.find_all('table')[0] #Find the first table on the web page
df_can = pd.read_html(str(table))[0] #Read the table into the object df_can
#Convert the data into three lists
postal_codes = df_can['Postal Code'].tolist() 
boroughs = df_can['Borough'].tolist()
neighborhoods = df_can['Neighborhood'].tolist()
#Convert the three lists into a Data Frame
df_final = pd.DataFrame({
    'Postal Code': postal_codes,
    'Borough': boroughs,
    'Neighborhood': neighborhoods
})

print(df_step_one.shape)
df_final.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The above code read the web page (scraped), converted the data into three lists and merged the lists into a Pandas dataframe (df_final). The next stage incolves doing some cleaning, specifically, removing the rows where Borough is not assigned. Then, if there are any Neighborhoods that are not assigned, they wil take the same value as Borough.


In [44]:
#Do some cleaning

#Remove the rows where the Borough is Not Assigned
df_final = df_step_one[df_final['Borough'] != 'Not assigned']
df_final.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [45]:
#There are no neighborhoods that are not assigned
df_final[df_final['Neighborhood'] == 'Not assigned'].head()

,Postal Code,Borough,Neighborhood


In [46]:
df_final.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M3H,North York,Downsview
freq,1,24,4


In [48]:
print('The shape fo the DataFrame is: ', df_final.shape)
df_final.head()

The shape fo the DataFrame is:  (103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
